In [30]:
! pip install git+https://github.com/arendjan/tmx-pico-aio.git@sensors
! pip install pynmea2

  Cloning https://github.com/arendjan/tmx-pico-aio.git (to revision sensors) to c:\users\arendjanvanhil\appdata\local\temp\pip-req-build-ms7l2iza
  Resolved https://github.com/arendjan/tmx-pico-aio.git to commit 3034f418e4fd0eff51de8fc52afedc203edda8b1
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/arendjan/tmx-pico-aio.git 'C:\Users\arendjanvanhil\AppData\Local\Temp\pip-req-build-ms7l2iza'
  Running command git checkout -b sensors --track origin/sensors
  branch 'sensors' set up to track 'origin/sensors'.
  Switched to a new branch 'sensors'


Create a board and connect your components:

In [31]:
import asyncio
import sys
import time
from tmx_pico_aio import tmx_pico_aio
import nest_asyncio
import pynmea2
nest_asyncio.apply() # for jupyter notebook asyncio stuff

loop = asyncio.get_event_loop()

# loop.create_task(stopBoard())

board = tmx_pico_aio.TmxPicoAio(loop=loop)

TelemetrixRpiPicoAio:  Version 1.5

Copyright (c) 2021 Alan Yorinks All Rights Reserved.

Opening all potential serial ports...
	COM13
Retrieving pico ID...
Pico Unique ID: [230, 97, 100, 8, 67, 88, 0, 0]
Telemetrix4RPiPico Version Number: 1.2


In [32]:
def the_callback(data):
    print(data)
    #do something



current_sentence = ""
items = []
def gps_callback(data): # each few ms, the uart buffer is checked and forwarded. This will be a few bytes, so it needs to be buffered and only when a complete NMEA sentence is received can it be parsed
    global current_sentence
    current_sentence += "".join(map(lambda d: chr(d), data)) # list of ASCII integers to string
    if "\n" in current_sentence: # only parse complete lines
        sentence = current_sentence.splitlines()
        if len(sentence) > 1: # when current_sentence ends with a newline, it will have only one element
            current_sentence = sentence[1]
        else:
            current_sentence = ""
        try: # first sentence is always broken
            msg = pynmea2.parse(sentence[0])
        except:
            return
        # print(msg.fields) # what is in the NMEA message
        # print(msg.name_to_idx) # What/how can we get those fields
        items.append(msg)
        if msg.sentence_type == "GGA":
            print(msg.latitude, msg.longitude)
        if msg.sentence_type == 'RMC':
            print("Speed", msg.spd_over_grnd)


In [33]:
async def addSensor(board):
    await asyncio.sleep(1)
    # await board.set_pin_mode_i2c(0, 4, 5)
    await asyncio.sleep(0.1)
    await board.sensors.add_gps(rx=5, tx=4, uart_channel=1, callback=gps_callback) # check the pinout to make sure it is the correct uart channel


loop.run_until_complete(addSensor(board))



In [34]:
async def run_till_end(board):
    # when to stop?
    await asyncio.sleep(10)
    await board.shutdown() # Otherwise next start will give you errors and you need to restart the kernel
    await asyncio.sleep(1)
    print("done")

loop.run_until_complete(run_till_end(board))

[0, 0, 36, 71, 80, 82, 77, 67, 44]
[0, 0, 44, 86, 44, 44, 44, 44, 44, 44, 44, 44]
[0, 0, 44, 44, 78, 42, 53, 51, 13, 10, 36, 71]
Speed None
[0, 0, 80, 86, 84, 71, 44, 44, 44, 44, 44]
[0, 0, 44, 44, 44, 44, 78, 42, 51, 48, 13, 10]
[0, 0, 36, 71, 80, 71, 71, 65, 44, 44, 44, 44]
[0, 0, 44, 44, 48, 44, 48, 48, 44, 57, 57]
[0, 0, 46, 57, 57, 44, 44, 44, 44, 44, 44, 42]
[0, 0, 52, 56, 13, 10, 36, 71, 80, 71, 83, 65]
0.0 0.0
[0, 0, 44, 65, 44, 49, 44, 44, 44, 44, 44]
[0, 0, 44, 44, 44, 44, 44, 44, 44, 44, 57, 57]
[0, 0, 46, 57, 57, 44, 57, 57, 46, 57, 57, 44]
[0, 0, 57, 57, 46, 57, 57, 42, 51, 48, 13]
[0, 0, 10, 36, 71, 80, 71, 83, 86, 44, 49, 44]
[0, 0, 49, 44, 48, 48, 42, 55, 57, 13, 10, 36]
[0, 0, 71, 80, 71, 76, 76, 44, 44, 44, 44]
[0, 0, 44, 44, 86, 44, 78, 42, 54, 52, 13, 10]
[0, 0, 36, 71, 80, 82, 77, 67, 44, 44, 86]
[0, 0, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44]
[0, 0, 78, 42, 53, 51, 13, 10, 36, 71, 80, 86]
Speed None
[0, 0, 84, 71, 44, 44, 44, 44, 44, 44, 44]
[0, 0, 44, 44, 78, 42, 

In [35]:
items

[<RMC(timestamp=None, status='V', lat='', lat_dir='', lon='', lon_dir='', spd_over_grnd=None, true_course=None, datestamp=None, mag_variation='', mag_var_dir='', mode_indicator='N')>,
 <VTG(true_track=None, true_track_sym='', mag_track=None, mag_track_sym='', spd_over_grnd_kts=None, spd_over_grnd_kts_sym='', spd_over_grnd_kmph=None, spd_over_grnd_kmph_sym='', faa_mode='N')>,
 <GGA(timestamp=None, lat='', lat_dir='', lon='', lon_dir='', gps_qual=0, num_sats='00', horizontal_dil='99.99', altitude=None, altitude_units='', geo_sep='', geo_sep_units='', age_gps_data='', ref_station_id='')>,
 <GSA(mode='A', mode_fix_type='1', sv_id01='', sv_id02='', sv_id03='', sv_id04='', sv_id05='', sv_id06='', sv_id07='', sv_id08='', sv_id09='', sv_id10='', sv_id11='', sv_id12='', pdop='99.99', hdop='99.99', vdop='99.99')>,
 <GSV(num_messages='1', msg_num='1', num_sv_in_view='00')>,
 <GLL(lat='', lat_dir='', lon='', lon_dir='', timestamp=None, status='V', faa_mode='N')>,
 <RMC(timestamp=None, status='V', 